# 04 | Functions Workshop

Build reusable engineering calculation functions with proper documentation and type hints.

## Exercise 1: Simple Function

Create a function to calculate beam self-weight.

In [1]:
def calculate_self_weight(length_m: float, mass_per_meter_kg: float) -> float:
    """
    Calculate total self-weight of a beam.
    
    Args:
        length_m: Beam length in meters
        mass_per_meter_kg: Linear mass in kg/m
    
    Returns:
        Total weight in kN
    """
    weight_kn = (length_m * mass_per_meter_kg * 9.81) / 1000
    return weight_kn

# Test
beam_weight = calculate_self_weight(8.0, 45.0)
print(f"Beam self-weight: {beam_weight:.2f} kN")

Beam self-weight: 3.53 kN


## Exercise 2: Function with Default Parameters

Calculate factored load with default load factors.

In [2]:
def calculate_factored_load(
    dead_load: float,
    live_load: float,
    dead_factor: float = 1.2,
    live_factor: float = 1.6
) -> float:
    """
    Calculate factored load combination.
    
    Args:
        dead_load: Dead load in kN
        live_load: Live load in kN
        dead_factor: Load factor for dead load (default 1.2)
        live_factor: Load factor for live load (default 1.6)
    
    Returns:
        Factored load in kN
    """
    return dead_load * dead_factor + live_load * live_factor

# Test with defaults
print(f"LRFD: {calculate_factored_load(50, 30):.1f} kN")

# Test with custom factors
print(f"Custom: {calculate_factored_load(50, 30, 1.4, 1.7):.1f} kN")

LRFD: 108.0 kN
Custom: 121.0 kN


## Exercise 3: Returning Multiple Values

Calculate multiple beam properties at once.

In [3]:
def analyze_simple_beam(load_kn_m: float, span_m: float) -> tuple[float, float, float]:
    """
    Analyze a simply supported beam with uniform load.
    
    Args:
        load_kn_m: Uniform load in kN/m
        span_m: Span length in meters
    
    Returns:
        Tuple of (total_load_kn, max_moment_kn_m, max_shear_kn)
    """
    total_load = load_kn_m * span_m
    max_moment = (load_kn_m * span_m ** 2) / 8
    max_shear = total_load / 2
    
    return total_load, max_moment, max_shear

# Unpack results
total, moment, shear = analyze_simple_beam(15.0, 6.0)

print(f"Total load: {total:.1f} kN")
print(f"Max moment: {moment:.1f} kN·m")
print(f"Max shear: {shear:.1f} kN")

Total load: 90.0 kN
Max moment: 67.5 kN·m
Max shear: 45.0 kN


## Exercise 4: Function with Conditional Logic

Classify concrete strength grade.

In [4]:
def classify_concrete_strength(fc_mpa: float) -> str:
    """
    Classify concrete by compressive strength.
    
    Args:
        fc_mpa: Compressive strength in MPa
    
    Returns:
        Strength classification
    """
    if fc_mpa < 20:
        return "Low strength"
    elif fc_mpa < 40:
        return "Normal strength"
    elif fc_mpa < 60:
        return "High strength"
    else:
        return "Ultra-high strength"

# Test
strengths = [15, 25, 40, 65]
for strength in strengths:
    classification = classify_concrete_strength(strength)
    print(f"{strength} MPa → {classification}")

15 MPa → Low strength
25 MPa → Normal strength
40 MPa → High strength
65 MPa → Ultra-high strength


## Exercise 5: Function Processing a List

Find the beam with maximum utilization ratio.

In [5]:
def find_critical_beam(beams: list[dict]) -> dict:
    """
    Find beam with highest utilization ratio.
    
    Args:
        beams: List of beam dictionaries with 'demand' and 'capacity'
    
    Returns:
        Beam dictionary with highest utilization
    """
    def get_utilization(beam: dict) -> float:
        return beam['demand'] / beam['capacity']
    
    return max(beams, key=get_utilization)

# Test data
project_beams = [
    {"id": "B1", "demand": 45, "capacity": 60},
    {"id": "B2", "demand": 72, "capacity": 80},
    {"id": "B3", "demand": 38, "capacity": 50},
]

critical = find_critical_beam(project_beams)
utilization = critical['demand'] / critical['capacity']

print(f"Critical beam: {critical['id']}")
print(f"Utilization: {utilization:.2%}")

Critical beam: B2
Utilization: 90.00%


## Exercise 6: Lambda Functions

Use lambda for quick transformations.

In [6]:
# Convert list of loads from kN to tons
loads_kn = [45, 60, 38, 72, 55]
loads_tons = list(map(lambda x: x / 9.81, loads_kn))

print("Loads in kN:", loads_kn)
print("Loads in tons:", [f"{t:.2f}" for t in loads_tons])

# Filter beams exceeding capacity
capacity = 50
overloaded = list(filter(lambda x: x > capacity, loads_kn))
print(f"Loads exceeding {capacity} kN:", overloaded)

Loads in kN: [45, 60, 38, 72, 55]
Loads in tons: ['4.59', '6.12', '3.87', '7.34', '5.61']
Loads exceeding 50 kN: [60, 72, 55]


## Challenge: Modular Program

Build a complete beam analysis utility using multiple functions.

In [7]:
def calculate_section_modulus(width_mm: float, depth_mm: float) -> float:
    """Calculate section modulus for rectangular section."""
    return (width_mm * depth_mm ** 2) / 6

def calculate_moment_capacity(
    section_modulus_mm3: float,
    allowable_stress_mpa: float
) -> float:
    """Calculate moment capacity."""
    return (section_modulus_mm3 * allowable_stress_mpa) / 1e6  # Convert to kN·m

def check_beam_adequacy(
    width_mm: float,
    depth_mm: float,
    applied_moment_kn_m: float,
    allowable_stress_mpa: float = 25
) -> dict:
    """
    Complete beam adequacy check.
    
    Returns:
        Dictionary with analysis results
    """
    section_modulus = calculate_section_modulus(width_mm, depth_mm)
    capacity = calculate_moment_capacity(section_modulus, allowable_stress_mpa)
    utilization = applied_moment_kn_m / capacity
    adequate = utilization <= 1.0
    
    return {
        "section_modulus_mm3": section_modulus,
        "capacity_kn_m": capacity,
        "demand_kn_m": applied_moment_kn_m,
        "utilization": utilization,
        "adequate": adequate
    }

# Run analysis
result = check_beam_adequacy(
    width_mm=200,
    depth_mm=400,
    applied_moment_kn_m=120
)

print("BEAM ANALYSIS RESULT")
print("=" * 40)
print(f"Section modulus: {result['section_modulus_mm3']:,.0f} mm³")
print(f"Moment capacity: {result['capacity_kn_m']:.1f} kN·m")
print(f"Applied moment:  {result['demand_kn_m']:.1f} kN·m")
print(f"Utilization:     {result['utilization']:.1%}")
print(f"Status:          {'✓ ADEQUATE' if result['adequate'] else '✗ INADEQUATE'}")

BEAM ANALYSIS RESULT
Section modulus: 5,333,333 mm³
Moment capacity: 133.3 kN·m
Applied moment:  120.0 kN·m
Utilization:     90.0%
Status:          ✓ ADEQUATE


## Key Takeaways

- Functions package logic into reusable units
- Docstrings and type hints improve readability
- Default parameters provide flexibility
- Return tuples for multiple values
- Break complex calculations into smaller functions
- Lambda functions work well for simple transformations